In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import wandb
import os
from PIL import Image
from torch.utils.data import Dataset

In [ ]:
sweep_config = {
    'method' : 'grid',
    'metric': {
        'name' : 'val_accuracy',
        'goal' : 'maximize'
    },
    'parameters' : {
        'batch_size' : { 'values' : [8, 16]},
        'learning_rate' : { 'values' : [0.001, 0.0001]},
        'hidden_nodes': {'values' : [32, 64]},
        'img_size' : {'values' : [224]},
        'epochs' : {'values': [5, 10]}
        }
}
sweep_id = wandb.sweep(sweep_config, project="5-Flower-Dataset-4")

Create sweep with ID: 7wva7j8c
Sweep URL: https://wandb.ai/fangselection123-happiest-minds-technologies/5-Flower-Dataset-4/sweeps/7wva7j8c


In [3]:
# =======================
# STEP 0: Initialize wandb
# =======================


# wandb.init(project="alexnet-flowers-v2", config={
#     "epochs": 10,
#     "batch_size": 16,
#     "learning_rate": 0.001,
#     "architecture": "alexnet",
#     "pretrained": True,
#     "input_size": 128
# })

# Shortcut to config values
# config = wandb.config

In [4]:


class CustomFlowersDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
        for class_name in class_names:
            class_dir = os.path. join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path =os.path.join(class_dir, img_name)
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[class_name])
        
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [2]:
class SimpleCNNModel(nn.Module):  
    def __init__(self):
        super(SimpleCNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels =384, out_channels= 256, kernel_size=3, stride=1, padding =1)
        self.pool3 = nn.MaxPool2d(kernel_size =3, stride=2)
        self.flatten = nn.Flatten()
        # Fully connected layers
        self.fc1 = nn.Linear(6*6*256, 4096)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer for regularization
        self.fc2 = nn.Linear(4096, 4096)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(4096, 5)  # Output layer for 5 classes

        # self.fc1 = nn.Linear(config.img_size * config.img_size * 3, config.hidden_nodes)
        # self.bn1 = nn.BatchNorm1d(config.hidden_nodes)
        # Output layer for 5 classes
        # self.dropout = nn.Dropout(p=0.2)
        # self.fc2 = nn.Linear(config.hidden_nodes, 5)

    def forward(self, x):
        x = self.conv1(x)
        # print(x.shape)
        x= self.relu(x)
        x=self.conv2(x)
        # print(x.shape)

        x = self.relu(x)
        x = self.pool1(x)
        # print(x.shape)

        x = self.conv3(x)
        # print(x.shape)

        x = self.relu(x)
        x = self.pool2(x)
        # print(x.shape)

        x=self.relu(x)
        x = self.conv4(x)
        # print(x.shape)

        x = self.relu(x)
        x = self.conv5(x)
        # print(x.shape)

        x= self.relu(x)
        x = self.pool3(x)
        # print(x.shape)

        x = self.flatten(x)
        # print(x.shape)

        x= self.fc1(x)
        # print(x.shape)

        x = self.relu(x)
        x= self.dropout(x)
        x = self.fc2(x)
        # print(x.shape)

        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x  # No softmax needed; use CrossEntropyLoss
model = SimpleCNNModel()
# print(model)

In [4]:
from torchsummary import summary

model = SimpleCNNModel()
summary(model, input_size=(3, 224, 224))  # adjust input size accordingly


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
            Conv2d-3          [-1, 256, 55, 55]         614,656
              ReLU-4          [-1, 256, 55, 55]               0
         MaxPool2d-5          [-1, 256, 27, 27]               0
            Conv2d-6          [-1, 384, 27, 27]         885,120
              ReLU-7          [-1, 384, 27, 27]               0
         MaxPool2d-8          [-1, 384, 13, 13]               0
              ReLU-9          [-1, 384, 13, 13]               0
           Conv2d-10          [-1, 384, 13, 13]       1,327,488
             ReLU-11          [-1, 384, 13, 13]               0
           Conv2d-12          [-1, 256, 13, 13]         884,992
             ReLU-13          [-1, 256, 13, 13]               0
        MaxPool2d-14            [-1, 25

In [6]:
# import torch
# model = SimpleCNNModel()
# dummy_input = torch.randn(1, 3, 224, 224)
# output = model(dummy_input)

In [7]:
import torchvision.models as models
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 5)
print(model)

/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
# import os

# path = "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/Data/flowers/train/.DS_Store"
# path = "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/Data/flowers/val/.DS_Store"

# # Remove .DS_Store if it exists
# if os.path.exists(path):
#     os.remove(path)
#     print(f"Deleted: {path}")
# else:
#     print(f"File does not exist: {path}")


In [9]:
def train():
    with wandb.init() as run:
        config= wandb.config
        train_dir="Data/flowers/train"
        test_dir="Data/flowers/val"
        transform = transforms.Compose([
            transforms.Resize((config.img_size, config.img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])



        train_dataset = CustomFlowersDataset(train_dir, transform=transform)
        test_dataset = CustomFlowersDataset(test_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


        class SimpleModel(nn.Module):
            def __init__(self):
                super(SimpleModel, self).__init__()
                self.flatten = nn.Flatten()
                self.fc1 = nn.Linear(config.img_size * config.img_size * 3, config.hidden_nodes)
                self.bn1 = nn.BatchNorm1d(config.hidden_nodes)
                # Output layer for 5 classes
                self.relu = nn.ReLU()
                self.dropout = nn.Dropout(p=0.2)
                self.fc2 = nn.Linear(config.hidden_nodes, 5)

            def forward(self, x):
                x = self.flatten(x)
                x = self.fc1(x)
                x= self.bn1(x)
                x = self.relu(x)
                x = self.dropout(x)
                x = self.fc2(x)
                return x  # No softmax needed; use CrossEntropyLoss
        model = SimpleCNNModel()
        # -----------------------------
        # Training Setup
        # -----------------------------
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=1e-5) # l2 regularization
        # shedule = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

        # -----------------------------
        # Training Loop
        # -----------------------------
        EPOCHS = 10
        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            for i, (images, labels) in enumerate(train_loader):
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                # L1 regularization (manual)
                # l1_lambda = 1e-5
                # l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
                # loss += l1_lambda * l1_loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # shedule.step()

                train_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                batch_correct = (preds == labels).sum().item()
                train_correct += batch_correct
                train_total += labels.size(0)
                # Print every 10 batches
                if(i + 1) % 10 == 0:
                    batch_acc = batch_correct / labels.size(0)
                    print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")

            train_accuracy = train_correct / train_total
            wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy})
            print(f"Epoch {epoch+1} Summary - Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

            # -----------------------------
            # # Evaluation (Optional)
            # # -----------------------------
        model.eval()
        test_correct = 0
        test_total = 0
        test_loss = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                test_correct += (preds == labels).sum().item()
                test_total += labels.size(0)
        test_accuracy = test_correct / test_total
        wandb.log({"test_loss": test_loss, "test_accuracy": test_accuracy})
        print(f"Test Accuracy: {test_correct / test_total:.4f}")


        




In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 1ymhh182 with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 32
wandb: 	img_size: 224
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: fangselection123 (fangselection123-happiest-minds-technologies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Batch 10/501] Loss: 1.5819, Batch Acc: 0.3750
[Batch 20/501] Loss: 1.6066, Batch Acc: 0.1250
[Batch 30/501] Loss: 1.6112, Batch Acc: 0.0000
[Batch 40/501] Loss: 1.5931, Batch Acc: 0.2500


In [ ]:
# print(len(train_dataset), len(test_dataset))
# print(len(train_loader), len(test_loader))
# for img, label in train_dataset:
#     print(img.shape, label)
#     break
# for img, label in test_dataset:
#     print(img.shape, label)
#     break
# for img, label in train_loader:
#     print(img.shape, label)
#     break   

# for img, label in test_loader:
#     print(img.shape, label)
#     break


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 62pjvwiu with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	hidden_nodes: 64
wandb: 	img_size: 224
wandb: 	learning_rate: 0.0001


Traceback (most recent call last):
  File "/tmp/ipykernel_91050/3860654642.py", line 64, in train
    outputs = model(images)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_91050/1075812509.py", line 42, in forward
    x= self.fc1(x)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-

wandb: ERROR Run 62pjvwiu errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 302, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_91050/3860654642.py", line 64, in train
wandb: ERROR     outputs = model(images)
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
wandb: ERROR     return self._call_impl(*args, **kwargs)
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
wandb: ERROR     return forward_call(*args, **kwargs)
wandb: ERROR   File "/tmp/ipykernel_91050/1075812509.py", line 42, in forward
wandb

Traceback (most recent call last):
  File "/tmp/ipykernel_91050/3860654642.py", line 64, in train
    outputs = model(images)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_91050/1075812509.py", line 42, in forward
    x= self.fc1(x)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-

wandb: ERROR Run a6qt7mb5 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 302, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_91050/3860654642.py", line 64, in train
wandb: ERROR     outputs = model(images)
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
wandb: ERROR     return self._call_impl(*args, **kwargs)
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
wandb: ERROR     return forward_call(*args, **kwargs)
wandb: ERROR   File "/tmp/ipykernel_91050/1075812509.py", line 42, in forward
wandb

Traceback (most recent call last):
  File "/tmp/ipykernel_91050/3860654642.py", line 64, in train
    outputs = model(images)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_91050/1075812509.py", line 42, in forward
    x= self.fc1(x)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-

wandb: ERROR Run pd6biagw errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 302, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_91050/3860654642.py", line 64, in train
wandb: ERROR     outputs = model(images)
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_impl
wandb: ERROR     return self._call_impl(*args, **kwargs)
wandb: ERROR   File "/home/ajit.kumar4@happiestminds.com/Documents/Drackula/ComputerVisionFromScratch/venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1762, in _call_impl
wandb: ERROR     return forward_call(*args, **kwargs)
wandb: ERROR   File "/tmp/ipykernel_91050/1075812509.py", line 42, in forward
wandb

In [ ]:
# -----------------------------
# Training Loop
# -----------------------------
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)
        # Print every 10 batches
        if(i + 1) % 10 == 0:
            batch_acc = train_correct / labels.size(0)
            print(f"[Batch {i+1}/{len(train_loader)}] Loss: {loss.item():.4f}, Batch Acc: {batch_acc:.4f}")


    train_accuracy = train_correct / train_total
    wandb.log({"epoch": epoch + 1, "train_loss": train_loss, "train_accuracy": train_accuracy})
    print(f"Epoch {epoch+1} Summary - Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

# -----------------------------
# # Evaluation (Optional)
# # -----------------------------
model.eval()
test_correct = 0
test_total = 0
test_loss = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Accuracy: {test_correct / test_total:.4f}")



NameError: name 'train_loader' is not defined